In [1]:
# tensorflow 버전 확인
!pip3 list | grep tensorflow

tensorflow                         1.15.2
tensorflow-estimator               1.15.1
tensorflow-gpu                     1.15.2
tensorflow-serving-api             1.15.0


In [2]:
# GPU 확인
!nvidia-smi

Fri Oct 29 23:21:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   31C    P0    27W / 250W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:00:07.0 Off |                    0 |
| N/A   29C    P0    26W / 250W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
# tokenizers 설치 및 google-bert clone
!pip install -q tokenizers
!git clone -q https://github.com/google-research/bert

# Tensorflow GPU 확인

In [5]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
print('GPU 사용여부: ', tf.test.is_gpu_available())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16278522054399609653
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11391216440161544829
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3641326972279770363
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16929045696832246212
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15752636007
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18026885225370774742
physical_device_desc: "device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:00:06.0, compute capability: 7.0"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 15752636007
locality {
  bus_id: 1
  links {
  }
}
incarnation:

# 토크나이저 학습

- 전체 데이터 사용
- vocab_size = 30000

In [5]:
from tokenizers import BertWordPieceTokenizer

In [6]:
tokenizer = BertWordPieceTokenizer(lowercase=False)
tokenizer.train(['./data/dataset.txt'], vocab_size=30000, show_progress=True, limit_alphabet=3000)
tokenizer.save_model('./data')

# 사전학습 데이터 생성하기

In [1]:
!mkdir ./shards 

In [2]:
# 학습 데이터셋 분할
# -a : 분할될 파일이름 접미사에 붙을 길이
# -l : 라인 수 기준으로 파일 분할
# -d : 분할될 파일이름과 저장될 파일이름 지정
!split -a 3 -l 4800000 -d ./data/dataset.txt ./shards/shard_

In [5]:
!mkdir ./pretraining_data

mkdir: cannot create directory ‘./pretraining_data’: File exists


In [6]:
!ls ./shards/ | xargs -n 1 -P 4 -I{} python bert/create_pretraining_data.py --input_file=./shards/{} --output_file=./pretraining_data/{}.tfrecord --vocab_file=./data/vocab.txt --do_lower_case=False --max_seq_length=128 --max_predictions_per_seq=20 --masked_lm_prob=0.15 --random_seed=12345 --dupe_factor=5







W1029 23:31:24.006067 139972354696960 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.
W1029 23:31:24.006064 140028057515776 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.
W1029 23:31:24.006067 140436707956480 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.






W1029 23:31:24.006265 140028057515776 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.
W1029 23:31:24.006260 139972354696960 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INF

# BERT MODEL CONFIG 파일 생성

In [3]:
import json

In [6]:
MODEL_DIR = "bert_model" 
tf.gfile.MkDir(MODEL_DIR)

In [7]:
bert_base_config = {
    "attention_probs_dropout_prob": 0.1, 
    "directionality": "bidi", 
    "hidden_act": "gelu", 
    "hidden_dropout_prob": 0.1, 
    "hidden_size": 768, 
    "initializer_range": 0.02, 
    "intermediate_size": 3072, 
    "max_position_embeddings": 512, 
    "num_attention_heads": 12, 
    "num_hidden_layers": 12, 
    "pooler_fc_size": 768, 
    "pooler_num_attention_heads": 12, 
    "pooler_num_fc_layers": 3, 
    "pooler_size_per_head": 128, 
    "pooler_type": "first_token_transform", 
    "type_vocab_size": 2, 
    "vocab_size": 30000,
    "model_type": "bert-base",
    "architectures": ["BertForMaskedLM"]
  }

with open("{}/config.json".format(MODEL_DIR), "w") as fo:
    json.dump(bert_base_config, fo, indent=2)

# BERT 사전학습

In [8]:
# 테스트로 1만번 학습 

!CUDA_VISIBLE_DEVICES=0 python bert/run_pretraining.py --input_file=../pretrain_test/src/pretraining_data/*.tfrecord --output_dir=./bert_model --do_train=True --do_eval=True --bert_config_file=./bert_model/config.json --train_batch_size=32 --max_seq_length=128 --max_predictions_per_seq=20 --num_train_steps=10000 --num_warmup_steps=10 --save_checkpoints_steps=1000 --learning_rate=2e-5




W1029 23:40:59.565044 139664882841344 module_wrapper.py:139] From bert/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1029 23:40:59.565279 139664882841344 module_wrapper.py:139] From bert/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1029 23:40:59.565592 139664882841344 module_wrapper.py:139] From /home/ubuntu/workspace/hate_speech/pretrain/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1029 23:40:59.566425 139664882841344 module_wrapper.py:139] From bert/run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1029 23:40:59.566731 139664882841344 module_wrapper.py:139] From bert/run_pretraining.py:418: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W1029 23:40:59.568123 139664882841344 module_wrappe